# Module 1, Section 1: Foundation Concepts

## Learning Goals

- Understand how LLMs interact with external data through tools
- Learn the fundamental building blocks: messages, tools, and tool calling
- See the manual tool calling loop in action
- Understand why we need agent abstractions (sets up Section 2)

## Overview

In this section, we'll build a simple customer support system for TechHub, an e-commerce electronics store. We'll start with the basics:
1. How to make LLM calls with messages
2. How to define tools that access external data (our database)
3. The manual tool calling loop (tedious but educational!)

By the end, you'll understand **why** agent frameworks exist - they automate the tedious parts we're about to implement manually.


## Setup

First, let's load our environment variables (API keys).


In [1]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

## 1. Basic LLM Call

Let's start simple - just calling an LLM with a message.


In [3]:
from langchain.chat_models import init_chat_model

# Configure model - change this to use different providers!
# Examples: "openai:gpt-4o-mini", "anthropic:claude-sonnet-4-5"
MODEL = "anthropic:claude-haiku-4-5"

# Initialize the model
llm = init_chat_model(MODEL)

# Simple string input
response = llm.invoke("What is LangChain in one sentence?")
response.pretty_print()

================================== Ai Message ==================================

LangChain is a framework for developing applications powered by language models that simplifies the process of chaining together multiple LLM calls and integrating them with external data sources and tools.


## 2. Working with Messages

LLMs work with **messages** that have roles:
- `SystemMessage`: Instructions/context for the LLM
- `HumanMessage`: User input
- `AIMessage`: LLM responses
- `ToolMessage`: Results from tool execution (we'll see this soon!)


In [4]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# Multi-turn conversation with messages
messages = [
    SystemMessage(
        content="You are a helpful customer support assistant for TechHub, an electronics store."
    ),
    HumanMessage(content="Hello!"),
    AIMessage(content="Hi! How can I help you today?"),
    HumanMessage(content="What's the status of order ORD-2024-0123?"),
]

response = llm.invoke(messages)
response.pretty_print()

================================== Ai Message ==================================

I'd be happy to help you check on that order! However, I don't have access to our order management system or customer account information.

To get the status of order **ORD-2024-0123**, I'd recommend:

1. **Check your email** - You should have received order confirmation and tracking emails
2. **Visit our website** - Log into your TechHub account and go to "My Orders" to see real-time status
3. **Call our support team** - They can look up your order with your order number and verify details
4. **Live chat with an agent** - Available on our website during business hours

Is there anything else I can help you with, or would you like information about our products and services?


**Notice the problem:** The LLM can't actually look up the order! It doesn't have access to our database.

This is where **tools** come in.


## 3. Defining Tools

Tools give LLMs the ability to interact with external systems. Let's create two simple tools that query our TechHub database.

The `@tool` decorator automatically:
- Extracts the function signature for the LLM
- Uses the docstring as the tool description
- Handles the input/output formatting


In [26]:
import sqlite3
from pathlib import Path
from langchain_core.tools import tool

# Path to our TechHub database
DB_PATH = Path("../../data/structured/techhub.db")


@tool
def get_order_status(order_id: str) -> str:
    """Look up the status of an order by order ID.

    Args:
        order_id: The order ID (e.g., "ORD-2024-0123")

    Returns:
        Formatted string with order status, shipped date, and tracking number.
    """
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    query = f"SELECT status, shipped_date, tracking_number FROM orders WHERE order_id = '{order_id}'"
    cursor.execute(query)
    result = cursor.fetchone()
    conn.close()

    if not result:
        return f"No order found with ID: {order_id}"

    status, shipped_date, tracking = result
    return f"Order {order_id}: Status={status}, Shipped={shipped_date or 'Not yet shipped'}, Tracking={tracking or 'N/A'}"


@tool
def get_product_price(product_name: str) -> str:
    """Get the current price of a product by name.

    Args:
        product_name: Product name to search for (e.g., "MacBook Air")

    Returns:
        Formatted string with product name, price, and stock status.
    """
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    query = (
        f"SELECT name, price, in_stock FROM products WHERE name LIKE '%{product_name}%'"
    )
    cursor.execute(query)
    result = cursor.fetchone()
    conn.close()

    if not result:
        return f"No product found matching: {product_name}"

    name, price, in_stock = result
    stock_status = "In Stock" if in_stock else "Out of Stock"
    return f"{name}: ${price:.2f} - {stock_status}"

Let's call of `get_product_price` tool

In [27]:
example_product = "MacBook Air"
result = get_product_price.invoke(example_product)
print(result)

MacBook Air M2 (13-inch, 256GB): $1199.00 - In Stock


Let's inspect how the `@tool` decorator parses the tool information into a `StructuredTool` object.


In [44]:
print("\n--- TOOL NAME ---")
print(get_product_price.name)
print("\n--- TOOL DESCRIPTION ---")
print(get_product_price.description)
print("\n--- TOOL ARGUMENTS ---")
print(get_product_price.args)


--- TOOL NAME ---
get_product_price

--- TOOL DESCRIPTION ---
Get the current price of a product by name.

Args:
    product_name: Product name to search for (e.g., "MacBook Air")

Returns:
    Formatted string with product name, price, and stock status.

--- TOOL ARGUMENTS ---
{'product_name': {'title': 'Product Name', 'type': 'string'}}


## 4. The Manual Tool Calling Loop

Now let's see how LLMs actually use tools! This happens in stages:

1. **Give the LLM access to tools** - Bind tools so the LLM knows what's available
2. **LLM decides when to call a tool** - Based on tool descriptions and the user's query
3. **LLM formats the function call** - Returns which tool to call and with what arguments (but doesn't execute it!)
4. **We manually execute the tool** - Run the actual function to get results
5. **Pass results back to the LLM** - So it can use them to generate a final answer

Let's see each stage in action!


### Step 1: Bind Tools to the Model

First, we tell the LLM what tools are available by "binding" them:


In [45]:
# Bind tools to the model - this tells the LLM what tools are available
tools = [get_order_status, get_product_price]
llm_with_tools = llm.bind_tools(tools)

### Step 2: LLM Decides to Call a Tool

Now let's give the LLM a query. It will decide which tool to call based on the descriptions:


In [61]:
# Ask about an order
query = "What's the status of order ORD-2024-0123?"

messages = [
    SystemMessage(content="You are a helpful customer support assistant."),
    HumanMessage(content=query),
]

# Call the LLM
response = llm_with_tools.invoke(messages)

response.pretty_print()

================================== Ai Message ==================================

[{'id': 'toolu_018zoVrJU8iLDWMqvUE6sjNw', 'input': {'order_id': 'ORD-2024-0123'}, 'name': 'get_order_status', 'type': 'tool_use'}]
Tool Calls:
  get_order_status (toolu_018zoVrJU8iLDWMqvUE6sjNw)
 Call ID: toolu_018zoVrJU8iLDWMqvUE6sjNw
  Args:
    order_id: ORD-2024-0123


### Step 3: Manually Execute the Tool

The LLM told us WHAT to call and WITH WHAT ARGUMENTS. Now **we** need to actually run the function:


In [62]:
# Extract the tool call information
tool_call = response.tool_calls[0]
tool_name = tool_call["name"]
tool_args = tool_call["args"]

print(f"Executing: {tool_name}(**{tool_args})")
print(f"{'='*60}\n")

# Manually execute the tool
if tool_name == "get_order_status":
    tool_result = get_order_status.invoke(tool_args)
elif tool_name == "get_product_price":
    tool_result = get_product_price.invoke(tool_args)

print(f"Tool Result: {tool_result}")

Executing: get_order_status(**{'order_id': 'ORD-2024-0123'})

Tool Result: Order ORD-2024-0123: Status=Delivered, Shipped=2024-12-07, Tracking=1Z999AA113527782


### Step 4: Pass Results Back to the LLM

Now we need to send the tool's result back to the LLM so it can generate a final answer for the user. Note that we use a `ToolMessage` type when adding tool result details back to the message history.


In [63]:
from langchain_core.messages import ToolMessage

# Add the AI's tool call to messages
messages.append(response)

# Create a ToolMessage with the result
tool_message = ToolMessage(
    content=str(tool_result),
    tool_call_id=tool_call["id"],  # Must match the ID from the tool call
)
messages.append(tool_message)
messages

[SystemMessage(content='You are a helpful customer support assistant.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="What's the status of order ORD-2024-0123?", additional_kwargs={}, response_metadata={}),
 AIMessage(content=[{'id': 'toolu_018zoVrJU8iLDWMqvUE6sjNw', 'input': {'order_id': 'ORD-2024-0123'}, 'name': 'get_order_status', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_01BSWheZ68JdgZqMdbEcxVJb', 'model': 'claude-haiku-4-5-20251001', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 758, 'output_tokens': 66, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-haiku-4-5-20251001', 'model_provider': 'anthropic'}, id='lc_run--44a9e7ac-b860-41dd-af9b-9b6153f59626-0', tool_calls=[{'name': 'get_order_status', 'args': {'order_id': 'ORD-2024

### Step 5: Get the Final Answer

Now the LLM has the tool result and can give a complete answer to the user:


In [67]:
# Call the LLM again with the tool result
final_response = llm_with_tools.invoke(messages)

final_response.pretty_print()

================================== Ai Message ==================================

Great! Here's the status of your order:

**Order ID:** ORD-2024-0123
- **Status:** Delivered ✓
- **Shipped Date:** December 7, 2024
- **Tracking Number:** 1Z999AA113527782

Your order has been successfully delivered! If you need any further assistance or have questions about your delivery, feel free to ask.


### Putting It All Together: The Loop

In real scenarios, the LLM might need to call multiple tools or loop several times. Let's create a function that automates this process:


In [73]:
from typing import Any


def run_agent_loop(user_query: str):
    """Run the complete tool calling loop."""

    messages = [
        SystemMessage(
            content="You are a helpful customer support assistant for TechHub."
        ),
        HumanMessage(content=user_query),
    ]

    print(f"\n{'='*60}")
    print(f"User: {user_query}")
    print(f"{'='*60}\n")

    # Keep looping until we get a final answer
    iteration = 0
    while True:
        iteration += 1

        # Call the LLM
        response = llm_with_tools.invoke(messages)

        # Check if LLM wants to use tools
        if not response.tool_calls:
            # No tools needed - we have the final answer!
            print(f"Final Answer: {response.content}\n")
            break

        # LLM wants to use tools
        print(
            f"[Iteration {iteration}] LLM is calling {len(response.tool_calls)} tool(s)..."
        )
        messages.append(response)

        # Execute each tool
        for tool_call in response.tool_calls:
            tool_name = tool_call["name"]
            tool_args = tool_call["args"]

            # Execute the tool
            if tool_name == "get_order_status":
                result = get_order_status.invoke(tool_args)
            elif tool_name == "get_product_price":
                result = get_product_price.invoke(tool_args)

            print(f"  → {tool_name}({list(tool_args.values())[0]}) = {result[:50]}...")

            # Add tool result to messages
            messages.append(
                ToolMessage(content=str(result), tool_call_id=tool_call["id"])
            )

        print()  # Blank line before next iteration

### Try It Out!

Now let's test our agent loop with different queries:


**Example 1: Simple order lookup**


In [74]:
run_agent_loop("What's the status of order ORD-2024-0123?")


User: What's the status of order ORD-2024-0123?

[Iteration 1] LLM is calling 1 tool(s)...
  → get_order_status(ORD-2024-0123) = Order ORD-2024-0123: Status=Delivered, Shipped=202...

Final Answer: Great news! Here's the status of your order:

**Order ID:** ORD-2024-0123
- **Status:** Delivered
- **Shipped Date:** December 7, 2024
- **Tracking Number:** 1Z999AA113527782

Your order has been successfully delivered. If you have any other questions or need further assistance, feel free to ask!



**Example 2: Product price lookup**


In [75]:
run_agent_loop("How much does the MacBook Air cost?")


User: How much does the MacBook Air cost?

[Iteration 1] LLM is calling 1 tool(s)...
  → get_product_price(MacBook Air) = MacBook Air M2 (13-inch, 256GB): $1199.00 - In Sto...

Final Answer: The **MacBook Air M2 (13-inch, 256GB)** is currently priced at **$1,199.00** and is **in stock**. 

Is there anything else you'd like to know about this product or any other items from TechHub?



**Example 3: Multiple tools in one query**


In [76]:
run_agent_loop(
    "What's the status of order ORD-2024-0123 and how much does a MacBook Air cost?"
)


User: What's the status of order ORD-2024-0123 and how much does a MacBook Air cost?

[Iteration 1] LLM is calling 2 tool(s)...
  → get_order_status(ORD-2024-0123) = Order ORD-2024-0123: Status=Delivered, Shipped=202...
  → get_product_price(MacBook Air) = MacBook Air M2 (13-inch, 256GB): $1199.00 - In Sto...

Final Answer: Great! Here's the information you requested:

**Order Status (ORD-2024-0123):**
- **Status:** Delivered
- **Shipped Date:** December 7, 2024
- **Tracking Number:** 1Z999AA113527782

**MacBook Air Price:**
- **Product:** MacBook Air M2 (13-inch, 256GB)
- **Price:** $1,199.00
- **Stock Status:** In Stock

Your order has already been delivered! If you have any other questions about your order or would like to purchase a MacBook Air, feel free to let me know!



## Key Takeaways

### What We Learned

1. **LLMs can't access external data** without tools
2. **Tools are functions** with clear schemas (name, description, arguments)
3. **The tool calling loop** is: LLM → tool call → execute → result → LLM → repeat until done
4. **This is tedious!** We had to:
   - Manually check for tool calls
   - Execute each tool ourselves
   - Format and append results
   - Manage the loop logic

### Why This Matters

Understanding the manual loop is crucial because:
- **Every agent framework does this** under the hood
- You can debug agent behavior by understanding what's happening
- You'll appreciate the abstractions in Section 2!

### What's Next

In **Section 2**, we'll replace all this manual code with `create_agent` - a simple abstraction that:
- Handles the tool calling loop automatically
- Adds memory (conversation history)
- Supports streaming responses
- Requires just a few lines of code!
